In [159]:
import pandas as pd
import numpy as np

train=pd.read_csv('adult.data',header=None,delim_whitespace=True)


In [185]:

class DesicionTree:
    
    def __init__(self,train,maxDepth=14,valNum=1000):
        self.maxDepth=maxDepth
        self.train=train.replace([' ?'],np.nan)
        self.train[15]=1        # Column 15 represents the weights of data
        self.val=self.train.sample(n=1000,random_state=0).dropna()
        #self.val=self.train.iloc[:valNum,:].dropna()
        self.dsctCol=[1,3,5,6,7,8,9,13]
        self.ctnsCol=[0,4,10,11,12]
        self.attributes=['age','workclass','fnlwgt','education','education-num',
            'marital-status','occupation','relationship','race','sex',
            'capital-gain','capital-loss','hours-per-week','native-country']
        self.dsctDict={}
        self.ctnsMean={}
        knownTrain=train.dropna()
        for col in self.dsctCol:
            self.dsctDict[self.attributes[col]]=knownTrain[col].value_counts()
        for col in self.ctnsCol:
            self.ctnsMean[self.attributes[col]]=knownTrain[col].mean()
        self.classDict=train[14].value_counts()
        
        
        
    def allSameClass(self,dataset,attributes):
        currentClass=''
        for index,row in dataset.iterrows():
            if row[14]!=currentClass:
                if currentClass!='':
                    return False
                else:
                    currentClass=row[14]

        return currentClass
    
    def allSameAttribute(self,dataset,attributes):

        sameAttribute={col:dataset.iloc[0][col] for col in attributes}
        
        for index,row in dataset.iterrows():
            for col in attributes:
                if row[col]!=sameAttribute[col]:
                    return False
                
        # show rarely happen
        return True
    
    def mostCommonClass(self,dataset):
        classDct=dataset[14].value_counts()       
        return classDct.index[0]
    
    
    def entropy(self,classDict):
        
        pClassDict=classDict.divide(classDict.sum())
        ent=0.0
        for className in self.classDict.index:
            if className in pClassDict.index and pClassDict[className]!=0: 
                ent+=pClassDict[className]*np.log2(pClassDict[className])
        return -1*ent
    
        
        
    def bestAttribute(self,dataset,attributes):
        infoGain=-float('inf')
        best_a,best_t=None,None
        temp_t=0
        
        
        for a in attributes:
            knownDataset=dataset.dropna()
            rho=knownDataset[15].sum()/dataset[15].sum() 
            knownEntropy=self.entropy(knownDataset[14].value_counts())
        
            if a in self.dsctCol:    # Discrete var
                
                subEntropy=0.0
                for dsctValue in self.dsctDict[self.attributes[a]].index:
                    tempDataset=knownDataset[knownDataset[a]==dsctValue]
                    subEntropy+=(tempDataset[15].sum()/knownDataset[15].sum())*\
                                self.entropy(tempDataset[14].value_counts())
                
                gain=rho*(knownEntropy-subEntropy)
            
            elif a in self.ctnsCol:    # Continuous var
                
                gain=-float('inf')
                tempCol=dataset[a].drop_duplicates().sort_values()
                if tempCol.size!=1:
                    tempT=[(tempCol.iloc[i]+tempCol.iloc[i+1])/2 
                           for i in range(tempCol.size-1)]
                else:
                    tempT=tempCol.tolist()
                    
                for t in tempT:
                    subEntropy=0.0
                    tempHiDataset=knownDataset[knownDataset[a]>t]
                    tempLoDataset=knownDataset[knownDataset[a]<=t]
                    subEntropy+=(tempHiDataset[15].sum()/knownDataset[15].sum()*\
                                self.entropy(tempHiDataset[14].value_counts()))
                    subEntropy+=(tempLoDataset[15].sum()/knownDataset[15].sum()*\
                                self.entropy(tempLoDataset[14].value_counts()))
                    if pd.isna(rho)==False:
                        tempGain=rho*(knownEntropy-subEntropy)
                    else:
                        tempGain=knownEntropy-subEntropy
                    
                    if tempGain>gain:
                        gain=tempGain
                        temp_t=t
            
            
            if gain>infoGain:
                infoGain,best_a=gain,a
                best_t=temp_t
      
        return best_a,best_t
    
    def prePruning(self,dataset,best_a,best_t):
        
        sub_nodes={}
        
        if best_a in self.dsctCol:    
            
            for dsctValue in self.dsctDict[self.attributes[best_a]].index:
                subDataset=dataset[dataset[best_a]==dsctValue]
                if subDataset.empty:
                    sub_nodes[self.attributes[best_a]+':'+dsctValue]=\
                        self.classDict.index[0]
                else:
                    sub_nodes[self.attributes[best_a]+':'+dsctValue]=\
                        self.mostCommonClass(subDataset)
            
        elif best_a in self.ctnsCol:
            
            # attributes.remove(best_a)
            subHiDataset=dataset[dataset[best_a]>best_t]
            
            if subHiDataset.empty:
                sub_nodes[self.attributes[best_a]+':>'+str(best_t)]=\
                    self.classDict.index[0]
            else:
                sub_nodes[self.attributes[best_a]+':>'+str(best_t)]=\
                    self.mostCommonClass(subHiDataset)
           
            subLoDataset=dataset[dataset[best_a]<=best_t]
            if subLoDataset.empty:
                sub_nodes[self.attributes[best_a]+':<='+str(best_t)]=\
                    self.classDict.index[0]
            else:
                sub_nodes[self.attributes[best_a]+':<='+str(best_t)]=\
                    self.mostCommonClass(subLoDataset)
        
        # Validation
        
        bfCommon=self.mostCommonClass(dataset)    # bf means before
        bfCorrect=0                               # af meas after
        afCorrect=0
        
        # Avoid useless iterations, when whether prune or not the prediction
        # is the same.
        noDiff=True
        for value in sub_nodes.values():
            if value!=bfCommon:
                noDiff=False
                break
        if noDiff:
            return False        
        
        for index,row in self.val.iterrows():
            if row[14]==bfCommon:
                bfCorrect+=1
            if row[14]==self.forward(row,sub_nodes):
                afCorrect+=1
        

        if bfCorrect>afCorrect:
            return bfCommon
        return False
        
    
    def treeGenerate(self,dataset,attributes,curDepth=0):
        
        
        # If remain dataset in the same class
        isSame=self.allSameClass(dataset,attributes)
        if isSame!=False:
            return isSame
        
        if len(attributes)==0 or self.allSameAttribute(dataset,attributes) or curDepth>self.maxDepth:
                return self.mostCommonClass(dataset)
        
        
        best_a,best_t=self.bestAttribute(dataset,attributes)
        # print("Best a and best t at this loop : "+str(best_a)+" "+str(best_t))
        
        
        isPruning=self.prePruning(dataset,best_a,best_t)
        if isPruning!=False:
            return isPruning
        
        
        sub_nodes={}
        unknownDataset=dataset[dataset[best_a].isnull()]
        
        if best_a in self.dsctCol:    # Choose a discrete node
            attributes.remove(best_a)  # Note : only in dsct we do this
            
            
            for dsctValue in self.dsctDict[self.attributes[best_a]].index:
                tempDataset=dataset[dataset[best_a]==dsctValue]
                r=tempDataset[15].sum()/dataset[15].sum()
                unknownDataset.loc[:,15]*=r
                
                subDataset=tempDataset.append(unknownDataset)
                if subDataset.empty:
                    sub_nodes[self.attributes[best_a]+':'+dsctValue]=\
                        self.classDict.index[0]
                else:
                    sub_nodes[self.attributes[best_a]+':'+dsctValue]=\
                        self.treeGenerate(subDataset,attributes,curDepth+1)
                unknownDataset.loc[:,15]/=r
            
        elif best_a in self.ctnsCol:
            
            # attributes.remove(best_a)
            # knownDataset=dataset.dropna()
            
            tempHiDataset=dataset.loc[dataset[best_a]>best_t]
            r=tempHiDataset[15].sum()/dataset[15].sum()
            unknownDataset.loc[:,15]*=r
            subHiDataset=tempHiDataset.append(unknownDataset)
            if subHiDataset.empty:
                sub_nodes[self.attributes[best_a]+':>'+str(best_t)]=\
                    self.classDict.index[0]
            else:
                sub_nodes[self.attributes[best_a]+':>'+str(best_t)]=\
                    self.treeGenerate(subHiDataset,attributes,curDepth+1)
                
            unknownDataset.loc[:,15]/=r
            
            tempLoDataset=dataset[dataset[best_a]<=best_t]
            r=tempLoDataset[15].sum()/dataset[15].sum()
            unknownDataset.loc[:,15]*=r
            subLoDataset=tempLoDataset.append(unknownDataset)
            if subLoDataset.empty:
                sub_nodes[self.attributes[best_a]+':<='+str(best_t)]=\
                    self.classDict.index[0]
            else:
                sub_nodes[self.attributes[best_a]+':<='+str(best_t)]=\
                    self.treeGenerate(subLoDataset,attributes,curDepth+1)
            
        # print(sub_nodes)      
        return sub_nodes
        
    def forward(self,row,sub_tree):
        # Reach the end of desicion tree
        if isinstance(sub_tree,str):
            return sub_tree
        
        first_key=next(iter(sub_tree))
        attribute_value=first_key.split(':')
        a=self.attributes.index(attribute_value[0])        
        
        if a in self.dsctCol:
            
            if pd.isna(row[a]):
                row[a]=self.dsctDict[self.attributes[a]].index[0]
            
            for key in sub_tree.keys():
                
                if row[a] in key:
                    # print(row[a])
                    return self.forward(row,sub_tree[key])
            
        elif a in self.ctnsCol:
            
            if pd.isna(row[a]):
                row[a]=self.ctnsMean[self.attributes[a]]
                
            for key in sub_tree.keys():
                if '>' in key:
                    hiKey=key
                elif '<' in key:
                    loKey=key
            
            t=float(hiKey.split('>')[-1])
            if row[a]<=t:
                return self.forward(row,sub_tree[loKey])
            return self.forward(row,sub_tree[hiKey])
        
    def predict(self,test,decisionTree):    # test is a dataframe
        test=test.replace([' ?'],np.nan)
        # print(test)
        correct=0
        for index,row in test.iterrows():
            res=self.forward(row,decisionTree)
            if row[14]==res+'.':
                correct+=1
        
        print("Correct:",correct)
        print("Total:",test.shape[0])
        print("Ratio:",correct/test.shape[0])
        
        


In [186]:
tree=DesicionTree(train)
decisionTree=tree.treeGenerate(tree.train,tree.dsctCol+tree.ctnsCol)

In [187]:
test=pd.read_csv('adult.test',header=None,sep=',')
tree.predict(test,decisionTree)

Correct: 13731
Total: 16281
Ratio: 0.8433757140224802


In [ ]:
print(decisionTree)
